In [6]:
import pandas as pd
import PyPDF2
import fitz  # PyMuPDF
import re

In [90]:
def get_text_from_questions(pdf_path, csv_path):
  # Open the PDF file
#   pdf_path = "data/edu-exam-p-sample-quest.pdf"
  doc = fitz.open(pdf_path)

  # Extract text starting from the second page
  text_content = ""
  for page_num in range(1, len(doc)):  # Skipping the first page (page_num starts from 0)
      text_content += doc.load_page(page_num).get_text()

  # Regex pattern to match question number, question, and multiple-choice answers
  pattern = r"(\d+)\.\s+(.+?)(?=\(A\))\(A\)\s*(.+?)\s*\(B\)\s*(.+?)\s*\(C\)\s*(.+?)\s*\(D\)\s*(.+?)\s*\(E\)\s*(.+?)(?=\d+\.\s|$)"
  matches = re.findall(pattern, text_content, re.DOTALL)

  # Organize the extracted data and clean Choice E of newline characters
  data = []
  for match in matches:
      question_number = match[0]
      question_text = match[1].strip()
      # choices = [choice.replace(" \n", "") for choice in match[2:]]
      choices = [choice.split(" ")[0] for choice in match[2:]]
      data.append([question_number, question_text] + choices)

  # Create DataFrame
  columns = ["Question Number", "Question", "Choice A", "Choice B", "Choice C", "Choice D", "Choice E"]
  df_questions = pd.DataFrame(data, columns=columns)

  # Save the DataFrame to a CSV file
#   csv_path = "data/cleaned_exam_questions.csv"
  df_questions.to_csv(csv_path, index=False)

  return df_questions


def get_text_from_answers(pdf_path, csv_path):
    # Open the PDF file
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        # Extract text from each page
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()

    # Regular expression to find question numbers and answers
    pattern = r"(\d+)\.\s*Solution:\s*([A-E])"
    matches = re.findall(pattern, text)

    # Create a DataFrame from the matches
    df = pd.DataFrame(matches, columns=["Question Number", "Answer"])

    # Save the DataFrame to a CSV file
    # csv_path = "data/exam-p-answers.csv"
    df.to_csv(csv_path, index=False)

    return df

In [92]:
if __name__ == "__main__":
    df_questions = get_text_from_questions("data/edu-exam-p-sample-quest.pdf", "data/exam-p-questions.csv")
    df_answers = get_text_from_answers("data/edu-exam-p-sample-sol.pdf", "data/exam-p-answers.csv")

### ...END